In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 생성형 AI 평가 : 나라명을 입력하면 그 나라의 제일 유명한 음식 레시피를 출력하는 복합 체인을 구현

- 지금까지의 예제 : 나라설명 → 나라이름 → 수도 (체인 두개 연결)
- 현재 평가 : 나라이름 → 그 나라에서 제일 유명한 음식(food_chain) → 레시피 출력(recipe_chain)

- exaone3.5:2.4b, llama3.1:405b(수능 언어영역 2위), llama3:70b
- llama3.2:3b

In [11]:
# import
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 모델
llm = ChatOllama(model='llama3.2:1b')
country_template = PromptTemplate(
    template = 'What is the the famouse food of {country}? Return the name of the food only.',
    input_variables = ['country'])
output_parsers = StrOutputParser()

# food chain
food_chain = country_template | llm | output_parsers
# food_chain.invoke({'country':'Korea'})

# 모델
food_template = PromptTemplate(
    template = 'What is the representative recipe for this {food}? Describe how to make it in 5 sentences. Whenever you print out a sentence, put the number in front of it. Briefly explain in around 80 characters per sentence. Without any explanation, print out the recipe right after the name of the food.',
    input_variables = ['food'])
output_parsers = StrOutputParser()

# recipe chain
recipe_chain = food_template | llm | output_parsers
# recipe_chain.invoke({'food':'Bulgogi'})

# final chain
final_chain = {'country':RunnablePassthrough()} | {'food':food_chain} | recipe_chain

In [12]:
final_chain.invoke('Japan')

'Sushi Rolls\n\n1. Prepare ingredients: 50g sushi rice, 10g pickled ginger, 10g wasabi paste, 5g soy sauce.\n2. Cook sushi rice according to package instructions.\n3. Cut nori sheets into desired size and shape.\n4. Lay nori sheet flat with side down on sushi mat.\n5. Spread 1-2 tablespoons of sushi rice onto nori sheet.\n6. Add pickled ginger, wasabi paste, and soy sauce on top of the rice.\n7. Roll up the sushi using sushi mat to form a tight roll.\n8. Slice into individual pieces.\n\nMaki Rolls'

In [14]:
print(final_chain.invoke('Spain'))

Paella

1. Heat a large paella pan or skillet over high heat. #Cooking
2. Add 2 tablespoons of olive oil and sauté chicken, shrimp, and chorizo until browned. #Meat
3. Add 1 onion, diced, and 2 cloves of garlic, minced, and cook until softened. #Vegetables
4. Add 1 cup of uncooked Arborio rice and stir to coat with oil and mix with ingredients. #Rice
5. Add 2 cups of chicken broth, 1 can of crushed tomatoes, and 1 teaspoon of smoked paprika. #Stocks
